In [1]:
import polars as pl
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import pyarrow as pyarrow

import glob 

In [ ]:
# Only run when loading files for the first time 

# Getting all decoded files 
decoded_files = glob.glob("pos_ct/*/*.decoded.hap*.txt")

# create list for storing df from each file 
dfs = []

# for each file extract pop and ID from the file name and add to each df 
for file in decoded_files:
    file_name = file.split('.')[0]
    ind_id = file_name.split('/')[2]
    pop = file_name.split('/')[1]
    
    df = pl.read_csv(file, has_header=True, separator='\t')
    
    # Adding ID and pop to each df
    df_1 = df.with_columns( # with _columns to add columns to a data frame 
        (pl.lit(ind_id).alias("ID")), # pl.lit returns literal value pl.alias to name a column
        (pl.lit(pop).alias("pop")),
        (pl.col("end") + 1000) # adding 1000 to the end coordinate to match fragment length 
        )
    # print(df_1.head(5))

    dfs.append(df_1) # adding df to the dfs list 

# concatenating dfs from the list 
decoded_df = pl.concat(dfs) 

# Saving to csv file to avoid runninng the code every time 
decoded_df.write_csv("decoded_df.csv", separator="\t", has_header=True)

In [2]:
# Read saved csv file containing all decoded files
decoded_df = pl.read_csv("decoded_df.csv", separator="\t", has_header=True)
decoded_df

chrom,start,end,length,state,mean_prob,snps,ID,pop
i64,i64,i64,i64,str,f64,i64,str,str
1,0,2132000,2132000,"""Human""",0.98143,33,"""HG00265""","""GBR"""
1,2132000,2170000,38000,"""Archaic""",0.68501,7,"""HG00265""","""GBR"""
1,2170000,3421000,1251000,"""Human""",0.9859,41,"""HG00265""","""GBR"""
1,3421000,3452000,31000,"""Archaic""",0.94517,23,"""HG00265""","""GBR"""
1,3452000,4526000,1074000,"""Human""",0.99345,27,"""HG00265""","""GBR"""
1,4526000,4553000,27000,"""Archaic""",0.9219,11,"""HG00265""","""GBR"""
1,4553000,5021000,468000,"""Human""",0.98596,13,"""HG00265""","""GBR"""
1,5021000,5083000,62000,"""Archaic""",0.971,37,"""HG00265""","""GBR"""
1,5083000,5920000,837000,"""Human""",0.99293,17,"""HG00265""","""GBR"""


In [ ]:
# Getting total fragments for each population 
decoded_df_GBR = decoded_df.filter(pl.col("pop") == "GBR")
print(decoded_df_GBR)

decoded_df_CHS = decoded_df.filter(pl.col("pop") == "CHS")
print(decoded_df_CHS)